In [8]:
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')

In [6]:
import pandas as pd

# Scrape hotel details from Bangalore using OYO Hotels

In [24]:
import time,re
def find(parent,tag,search='text'):
    try:
        if search == 'text':
            return parent.find_element_by_css_selector(tag).text
        elif search == 'price':
            try:
                return re.findall("(^. .{3}) .*",parent.find_element_by_css_selector(tag).text)[0]
            except:
                return re.findall("(^. .{4}) .*",parent.find_element_by_css_selector(tag).text)[0]
        elif search == 'aprice':
            try: 
                return re.findall("^. .{3} (. .{4}) .*",parent.find_element_by_css_selector(tag).text)[0]
            except:
                return re.findall("^. .{4} (. .{4}) .*",parent.find_element_by_css_selector(tag).text)[0]
        elif search == 'rating':
            return parent.find_element_by_css_selector(tag).text.split(' ')[0]
        else:
            return parent.find_element_by_css_selector(tag).get_attribute(search)
    except:
        return None
df = pd.DataFrame()
browser.get("https://www.oyorooms.com/oyos-in-bangalore")
while True:
    time.sleep(2)
    hotelstag = browser.find_elements_by_css_selector('div.newHotelCard')
    for hotel in hotelstag:
        cur_row = {'HotelName': find(hotel,'span.newHotelCard__hotelName'),
                'Location':find(hotel,'span.newHotelCard__hotelAddress'),
                'Actual Price':find(hotel,'div.newHotelCard__pricing','aprice'),
                'Discounted Price':find(hotel,'div.newHotelCard__pricing','price'),
                'Hotel rating':find(hotel,'span.hotelRating__value'),
                'Total number of ratings':find(hotel,'div.hotelRating__count','rating'),
                'Source of the image':find(hotel,'img.carousal__image','src')
              }
        
        df = df.append(cur_row,ignore_index=True)
    break
    #To press next button
    try:
        atag = browser.find_elements_by_css_selector('a.disabled')[-1]
        atag.get_attribute('div.btn-next')
        atag.click()
    except Exception as e:
        print(e)
        break
df.head()

,Actual Price,Discounted Price,Hotel rating,HotelName,Location,Source of the image,Total number of ratings
0,₹ 3032,₹ 726,4.6,Edition O 30089 1st Cross Road Cottonpet,"15, 1st Cross Road K.V Lane, Cottonpete Cross,...",https://images.oyoroomscdn.com/uploads/hotel_i...,57
1,None,None,None,None,None,None,None
2,₹ 2242,₹ 1121,None,Capital O 36472 Hotel Raj Vista Suites,"No.4 & 4A, Hesarghatta Main Rd, Besides NRR Ho...",https://www.oyorooms.com/images/transparent-c4...,None
3,₹ 2220,₹ 999,4.8,OYO 29620 New Champion Residency,"No 390, Old Madras Road, ILD Extension, 4th Ma...",https://www.oyorooms.com/images/transparent-c4...,34
4,₹ 2640,₹ 964,3.8,Capital O 1309 Anandsagar Inn,"S.C. Road, Netkalappa Circle Basavanagudi, Ban...",https://www.oyorooms.com/images/transparent-c4...,28


# Scrape data science articles from Medium Portal

In [25]:
browser.get('https://medium.com/search?q=data%20science')
parent = browser.find_element_by_css_selector('div.js-postList')
parent

<selenium.webdriver.remote.webelement.WebElement (session="78250c05b40f5f03af45b14346aea366", element="0.8229243904930839-1")>

In [26]:
child = parent.find_elements_by_css_selector('div.js-block')
article = pd.DataFrame()
for tag in child:
    cur_row = {
        'Article title' : tag.find_element_by_css_selector('h3.graf--title').text,
        'Author' : tag.find_element_by_css_selector('a.ds-link').text,
        'URL of the article' : tag.find_element_by_css_selector('a.button').get_attribute('href'),
        'Date of the articles': tag.find_element_by_css_selector('time').text
    }
    article = article.append(cur_row,ignore_index=True)
article.head()

,Article title,Author,Date of the articles,URL of the article
0,How To Learn Data Science If You’re Broke,Harrison Jansma,"Sep 15, 2018",https://towardsdatascience.com/how-to-learn-da...
1,"If you want to learn Data Science, start with ...",David Venturi,"Sep 26, 2016",https://medium.freecodecamp.org/if-you-want-to...
2,I ranked every Intro to Data Science course on...,David Venturi,"Jan 25, 2017",https://medium.freecodecamp.org/i-ranked-all-t...
3,Data Science and Machine Learning Interview Qu...,George Seif,"Apr 25, 2018",https://towardsdatascience.com/data-science-an...
4,How to Build a Data Science Portfolio,Michael Galarnyk,"Jul 8, 2018",https://towardsdatascience.com/how-to-build-a-...


In [28]:
for link in article.iterrows():
    browser.get(link[1]['URL of the article'])
    ptags = browser.find_elements_by_css_selector('p')
    text = ''
    for tag in ptags:
        text += ' ' + tag.text
    filename =link[1]['Author'] + '.txt'
    file = open(filename,'w',encoding='utf-8')
    file.write(text)
print('Text files created')

Text files created
